# Minimum Wage and Inflation, Discourse

Final Project for CUNY DATA608 by Chris G. Martin, 05/21/2016

### **NOTE: This is Part Two of Three**

This part includes webscraping for the raw data and explainations as to the how and why. Part One includes the data analysis and discussions performed in an RMarkdown file, while Part Three includes an interactive d3 visualization of the data.

In [1]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd
from itertools import chain
import csv

After importing the required packages, I've created a function below to scrape the page for the necessary information:

In [2]:
def get_wage_data(urlname):
    """
    scrapes the DOL website table
    """
    sock = urllib.urlopen(urlname).read()
    soup = BeautifulSoup(sock, 'html.parser')
    content = soup.find("div", {"id" : "content"})
    #set parameters for dataframe
    states = []
    col1 = []
    headertable = content.findAll('table', {"class" : "minwage"})
    rows = content.findAll('tr')
    for row in rows:
        states.append(row.findAll('td'))
        col1.append(row.find_next('td'))
#    table = content.findAll('table', {"class" : "minwage"})
#    rows = table.findAll('th', {"class" : "minwage"})
#    for row in rows:
#        states.append(row.find('td'))
    #songnames = pd.DataFrame(songnames[:200])
    #songnames = songnames.astype(str)
    #extract URLs in Source column
    #songnames['source'] = songnames[0].str.extract('(\".*?\")', expand=True)
    #songnames['source'] = songnames['source'].str.replace('(\"\/)', 'http://www.lyricsfreak.com/')
    #songnames['source'] = songnames['source'].str.replace('(\")', '')
    #extract title in title column
    #songnames['Title'] = songnames[0].str.extract('(title\=\".*?\")', expand=False)
    #songnames['Title'] = songnames['Title'].str.replace('(title\=\")', '')
    #songnames['Title'] = songnames['Title'].str.replace('(Lyrics\")', '')
    #drop initial column
    return states
#.drop([0], axis=1)

dolwagehistoryurl = 'https://www.dol.gov/whd/state/stateMinWageHis.htm'
dolwagehistorytable = get_wage_data(dolwagehistoryurl)

Unfortunately, the table is extremely messy and ot useable in it's current state. As an example:

In [3]:
dolwagehistorytable[1]

[<td>Federal (FLSA)</td>,
 <td>$1.15 &amp; $1.60</td>,
 <td>$1.30 &amp; $1.60</td>,
 <td>$1.60 </td>,
 <td>$2.20 &amp; $2.30</td>,
 <td>$2.90 </td>,
 <td>$3.10 </td>,
 <td>$3.35 </td>]

In [4]:
dolwagehistorytable

[[],
 [<td>Federal (FLSA)</td>,
  <td>$1.15 &amp; $1.60</td>,
  <td>$1.30 &amp; $1.60</td>,
  <td>$1.60 </td>,
  <td>$2.20 &amp; $2.30</td>,
  <td>$2.90 </td>,
  <td>$3.10 </td>,
  <td>$3.35 </td>],
 [<td>Alabama</td>,
  <td>...</td>,
  <td>...</td>,
  <td>...</td>,
  <td>...</td>,
  <td>...</td>,
  <td>...</td>,
  <td>...</td>],
 [<td>Alaska</td>,
  <td>2.10</td>,
  <td>2.10</td>,
  <td>2.10</td>,
  <td>2.80</td>,
  <td>3.40</td>,
  <td>3.60</td>,
  <td>3.85</td>],
 [<td>Arizona</td>,
  <td>18.72 \r\n      - 26.40/wk(b)</td>,
  <td>18.72 \r\n      - 26.40/wk(b)</td>,
  <td>18.72-26.40/wk(b)</td>,
  <td>\xa0...</td>,
  <td>...</td>,
  <td>...</td>,
  <td>...</td>],
 [<td>Arkansas</td>,
  <td>1.25/day(b)</td>,
  <td>1.10</td>,
  <td>1.20</td>,
  <td>1.90</td>,
  <td>2.30</td>,
  <td>2.55</td>,
  <td>2.70</td>],
 [<td>California</td>,
  <td>1.65(b)</td>,
  <td>1.65(b)</td>,
  <td>1.65(b)</td>,
  <td>2.00</td>,
  <td>2.90</td>,
  <td>2.90</td>,
  <td>3.35</td>],
 [<td>Colorado</td>,
  <td

In [6]:
len(dolwagehistorytable)-55

226

In [10]:
dolwagehistorytable[223]

[<td>U.S. \r\n      Virgin Islands</td>,
 <td>4.30-6.15(g,j)</td>,
 <td>4.30-6.15(j)</td>,
 <td>4.30-6.15(j)</td>,
 <td><strong>4.30-7.25(j)</strong></td>,
 <td>4.30-7.25(j)</td>,
 <td>4.30-7.25(j)</td>,
 <td>4.30-7.25(j)</td>]

As you can see, the first 'row' is actually a tuple containing 7 rows. This won't work. Instead, what we'll have to do is clean it. Since my skills in R is more handy in this situation, I'll export the result into a csv file and import it into R for clean-up.

In [23]:
#iterate through the table and change to a list
#then make it a dataframe
dolwages = pd.DataFrame(list(chain.from_iterable(dolwagehistorytable)))
dolwages.to_csv('dolwages.csv', index=False, header=False)